In [ ]:

!pip3 install transformers optimum
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

!pip install -qU \
sentence-transformers==2.2.2 \
accelerate==0.21.0 \
einops==0.6.1 \
langchain \
#xformers==0.0.20 \



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.4/403.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 18.9 MB/s eta 0:00:00
    

In [ ]:
!pip install "weaviate-client==3.*"
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 8.0 MB/s eta 0:00:00


In [ ]:
! pip install langchain auto-gptq streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.1 MB/s eta 0:00:00
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [ ]:
%%writefile app.py
import streamlit as st
import transformers
import streamlit as st
from torch import cuda, bfloat16
from langchain.vectorstores import Weaviate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
import weaviate
import sys
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from auto_gptq import exllama_set_max_input_length
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.schema.prompt_template import BasePromptTemplate
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

def logout():
    st.balloons()
    st.warning("Logged out successfully!")
    st.session_state.logged_in = False

# Initialize session state
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
@st.cache_resource
def pipeln():
   device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
    model_name_or_path= "TheBloke/zephyr-7B-beta-GPTQ"

    model_name = "BAAI/bge-base-en-v1.5"
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

    embedding_function = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cuda'},
        encode_kwargs=encode_kwargs,

    )

    loader = CSVLoader(file_path="/content/cleaned_data (3).csv")
    documents = loader.load()
    WEAVIATE_URL = "https://hackaphasia-dataset-2-ctmvkrah.weaviate.network"
    WEAVIATE_API_KEY = "s0NO2IvRGawZN1J3LmMzX61f0WUygAfY70nC"

    weaviate_client = weaviate.Client(
        url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
    )
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embedding = embedding_function
    weaviate_vectorstore = Weaviate.from_documents(texts, embedding, client=weaviate_client, by_text=False)
    weaviate_retriever = weaviate_vectorstore.as_retriever(search_kwargs={"k":10})




    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                device_map="auto",
                                                trust_remote_code=False,
                                                revision="main")
    model = exllama_set_max_input_length(model, max_input_length=7000)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    generate_text = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.5,
    repetition_penalty=1.1,
    top_p=0.95,
    top_k=1,

    )
    llm = HuggingFacePipeline(pipeline=generate_text)
    prompt_template = """Use the following pieces of context to answer the question at the end accurately. If you don't know the answer strictly say that "I don't know", don't try to make up any answer and
                     If context and question doesn't relate then strictly do not give answer, just say that "I don't know" and stop at that point.
    {context}

    Question: {question}
    Answer :"""
    CONV_PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    chat = ConversationalRetrievalChain.from_llm(llm, retriever=weaviate_retriever, memory=memory,combine_docs_chain_kwargs={"prompt":CONV_PROMPT})

    return chat,weaviate_retriever

if not st.session_state.logged_in:
    st.subheader("Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    login_button = st.button("Login")

    if login_button:
        # Use a secure authentication method here
        # ...

        if username == "User" and password == "user@123":
            st.session_state.logged_in = True
            st.success("Login successful!")

if st.session_state.logged_in:


# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"

    def main():

        menu = ['Home', 'About', 'Chat']
        ch = st.sidebar.selectbox('Menu', menu)
        if ch == 'Home':
            st.title("Welcome to :blue[SciBot]")
            st.markdown("<h1 style='text-align: center;'>-Welcome to <span style='color: lightblue;'>SciBot🤖-</span></h1>", unsafe_allow_html=True)
            st.markdown("<h4>Specific Requirements: </h4>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Welcome to SciBot!</span></div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Get all your queries related to patent resolved in minutes </span></div>", unsafe_allow_html=True)

            st.subheader('Home page')
        if ch == 'About':
            st.markdown("<h1 style='text-align: center;'>-ABOUT-</h1>", unsafe_allow_html=True)

            st.markdown("<div style='text-align: center;'><span style='font-size: 22px; font-weight: 700;'>Problem Statement: </span>Chatbot to explore science and technologies</h3>", unsafe_allow_html=True)
            st.markdown("<div style='text-align: justify; margin-top: 2vh;'>Traditional keyword-based patent searches face limitations with complex jargon and lengthy documents, hindering semantic understanding. To overcome this, advanced information retrieval systems leverage Natural Language Processing and machine learning. Semantic search algorithms analyze word relationships, enhancing relevance. Machine learning models, like deep learning architectures, categorize, summarize, and extract key patent information from extensive texts. User interfaces with natural language capabilities improve user experience, allowing intuitive queries. Overall, integrating NLP, machine learning, and user-friendly interfaces addresses keyword search limitations, promising more efficient and accurate patent information retrieval.</div>", unsafe_allow_html=True)
            st.markdown("<h4>Specific Requirements: </h4>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Semantic Understanding: </span>The chatbot must go beyond keyword matching, understanding the semantic context of user queries.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Information Extraction: </span>Capable of scanning through patent text and extracting relevant information in response to the queries.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Interactivity: </span>Engage with users in a conversational manner, allowing for follow-up questions and clarification.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Summarization: </span>Provide concise summaries of extracted information, maintaining accuracy and relevance.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>User Query Learning: </span>Learn from user interactions to improve response accuracy over time.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Scalability: </span>Efficiently handle a large volume of articles and simultaneous user queries.</div>", unsafe_allow_html=True)

        if ch == 'Chat':
            st.title("Chat Page")

            with st.sidebar:
                if st.button("Refresh"):
                    st.experimental_rerun()

            if "messages" not in st.session_state:
                st.session_state.messages = []


            for message in st.session_state.messages:
                with st.chat_message(message["role"]):
                    st.markdown(message["content"])

            if prompt := st.chat_input("Enter text: "):

                with st.chat_message("user"):
                    st.markdown(prompt)
                st.session_state.messages.append({"role" :"user", "content":prompt})
                chat,weaviate_retriever = pipeln()
                # Load sentence transformer model
                model_cos_sim = SentenceTransformer('all-MiniLM-L6-v2')


                docs = weaviate_retriever.get_relevant_documents(prompt)
                ans = ""
                for document in docs:
                    ans += f"{document.metadata.get('title')} {document.metadata.get('URL')} {document.page_content}"+"\n"
                answer_samples = ans

                query_embedding = model_cos_sim.encode(prompt, convert_to_tensor=True)
                answer_embedding = model_cos_sim.encode(answer_samples, convert_to_tensor=True)

                score = util.cos_sim(query_embedding, answer_embedding)

                if score > 0.1:
                  ans = ""
                  docs = weaviate_retriever.get_relevant_documents(prompt)
                  for document in docs:
                      ans += f"\n{document.metadata.get('title')}\t {document.metadata.get('URL')} \n{document.page_content} \n\n"
                  ans = ans.replace("None None", "")
                  answer_for_retreive = ans
                else:
                  answer_for_retreive = ""
                # Get and display chatbot response

                bot_response = chat({"question": prompt})['answer'] +"\n"+ answer_for_retreive
                response=f"SciBot: {bot_response}"
                with st. chat_message("assistant"):
                    st.markdown(response)
                st.session_state.messages.append({"role": "assistant", "content": response})

    if __name__ == '__main__':
        main()

    # Implement chatbot functionality here using NLP libraries

if st.sidebar.button("Logout"):
  logout()



Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pip install pyngrok==4.1.1
from pyngrok import ngrok
# !ngrok config add-authtoken 2ZVwALoImirOuZfwbb6dws6KO0G_829DDbQD1RL1TPbr5b1sA


  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=ac560220b20136e2a0022f49e00b70bed581495a4c47aec55f9f09c2caa3223a
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 7.0.3
    Uninstalling pyngrok-7.0.3:
      Successfully uninstalled pyngrok-7.0.3


In [ ]:
!killall ngrok

ngrok: no process found


In [ ]:
!ngrok config add-authtoken 2ZWpnhJCzFUeR5rNK6UnGOMTKdZ_31rkpdRxeRPoP9Q3GU4hD


NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
!ngrok authtoken 2ZWpnhJCzFUeR5rNK6UnGOMTKdZ_31rkpdRxeRPoP9Q3GU4hD
#!ngrok authtoken NGROK_Token
public_url = ngrok.connect(port='1090
')
print(public_url)

SyntaxError: ignored